In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="P1tOolfxuLaCFYy4PyEb")
project = rf.workspace("naspadsupremasi").project("citrus-disease-cjcbq")
version = project.version(7)
dataset = version.download("coco-segmentation")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.7 MB/s eta 0:00:00
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to citrus-disease-7 in coco-segmentation:: 100%|██████████| 2004/2004 [00:00<00:00, 3991.92it/s]


In [ ]:
!mkdir /content/citrus-disease-7/mask

In [ ]:
import json
import numpy as np
import cv2
from pycocotools import mask as maskUtils
from tqdm import tqdm
import os

coco_json_path = '/content/citrus-disease-7/train/_annotations.coco.json'
img_dir = '/content/citrus-disease-7/train'
mask_output_dir = '/content/citrus-disease-7/mask'


with open(coco_json_path) as f:
    coco = json.load(f)

# Map image_id to file_name
id_to_filename = {img['id']: img['file_name'] for img in coco['images']}

# Group annotations by image_id
from collections import defaultdict
ann_dict = defaultdict(list)
for ann in coco['annotations']:
    ann_dict[ann['image_id']].append(ann)

cat_id_to_class = {
    0: 1,  # lesion
    1: 2,  # fruit
    2: 1   # lesion
}

class_priority = {
    0: 2,  # lesion
    1: 1,  # fruit
    2: 2   # lesion
}

for img_id, anns in tqdm(ann_dict.items()):
    img_info = next(item for item in coco['images'] if item['id'] == img_id)
    height, width = img_info['height'], img_info['width']
    mask = np.zeros((height, width), dtype=np.uint8)

    # Sort anns by priority (fruit first, lesion last)
    anns = sorted(anns, key=lambda x: class_priority.get(x['category_id'], 0))

    for ann in anns:
        segmentation = ann['segmentation']
        category_id = ann['category_id']
        class_id = cat_id_to_class.get(category_id, 0)

        if isinstance(segmentation, list):
            for seg in segmentation:
                pts = np.array(seg).reshape(-1, 2).astype(np.int32)
                if pts.shape[0] >= 3:
                    cv2.fillPoly(mask, [pts], color=class_id)
        else:
            rle = maskUtils.frPyObjects(segmentation, height, width)
            m = maskUtils.decode(rle)
            mask[m > 0] = class_id

    filename = id_to_filename[img_id].replace('.jpg', '.png')
    cv2.imwrite(os.path.join(mask_output_dir, filename), mask * 100)



100%|██████████| 2000/2000 [00:11<00:00, 179.73it/s]


In [ ]:
import os

folder_path = '/content/citrus-disease-7/mask'  # change to your folder path
file_count = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])

print(f"Number of files: {file_count}")


Number of files: 2000


In [ ]:
#delete json

In [ ]:
!rm -r /content/citrus-disease-7/train/_annotations.coco.json

In [ ]:
import os
from glob import glob
from os.path import basename
import cv2

image_paths = sorted(glob('/content/citrus-disease-7/train/*.jpg'))

# Build mask paths: use actual if exists, else create an empty placeholder later
mask_dir = '/content/citrus-disease-7/mask'
mask_paths = []

for img_path in image_paths:
    name = basename(img_path).replace('.jpg', '')
    mask_path = os.path.join(mask_dir, f'{name}.png')`

    if os.path.exists(mask_path):
        mask_paths.append(mask_path)
    else:
        # Mark empty with a placeholder (we’ll generate in dataset class)
        mask_paths.append(None)


In [ ]:
!pip install -q segmentation-models-pytorch albumentations

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.2 MB/s eta 0:00:00


In [ ]:
import os
import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2


In [ ]:
class SegmentationDataset(Dataset):
    def __init__(self, images_dir, masks_dir, transform=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform
        self.images = sorted(os.listdir(images_dir))
        self.masks = sorted(os.listdir(masks_dir))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = np.array(Image.open(os.path.join(self.images_dir, self.images[idx])).convert("RGB"))
        mask = np.array(Image.open(os.path.join(self.masks_dir, self.masks[idx])))

        # Map unique pixel values to class indices
        mask = np.where(mask == 100, 1, mask)
        mask = np.where(mask == 200, 2, mask)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask'].long()

        return image, mask


In [ ]:
train_transform = A.Compose([
    A.Resize(256, 256),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])
from torch.utils.data import random_split

# Full dataset
full_dataset = SegmentationDataset(
    images_dir='/content/citrus-disease-7/train',
    masks_dir='/content/citrus-disease-7/mask',
    transform=train_transform
)

# Split sizes
val_percent = 0.2
val_size = int(val_percent * len(full_dataset))
train_size = len(full_dataset) - val_size

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])


In [ ]:
model = smp.Unet(
    encoder_name="vgg16_bn",
    encoder_weights="imagenet",
    in_channels=3,
    classes=3,  # 3 classes (0, 1, 2)
)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/554M [00:00<?, ?B/s]

Unet(
  (encoder): VGGEncoder(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ReLU(inplace=True)
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): ReLU(inplace=True)
      (13): MaxPool2d(kernel_size=2, stride=2, padding=0

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=2)


In [ ]:
def compute_iou(preds, labels, num_classes):
    ious = []
    preds = preds.view(-1)
    labels = labels.view(-1)

    for cls in range(num_classes):
        pred_inds = preds == cls
        label_inds = labels == cls
        intersection = (pred_inds & label_inds).sum().item()
        union = (pred_inds | label_inds).sum().item()
        if union == 0:
            ious.append(float('nan'))  # Ignore class if not present
        else:
            ious.append(intersection / union)

    return ious
def evaluate_iou(model, dataloader, num_classes, device):
    model.eval()
    iou_list = []

    with torch.no_grad():
        for images, masks in dataloader:
            images = images.to(device)
            masks = masks.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            ious = compute_iou(preds, masks, num_classes)
            iou_list.append(ious)

    # Average over all batches
    iou_array = np.array(iou_list)
    mean_iou = np.nanmean(iou_array, axis=0)  # per-class IoU
    miou = np.nanmean(mean_iou)              # mean IoU across classes
    return mean_iou, miou


In [ ]:

for epoch in range(25):
    model.train()
    total_loss = 0
    for images, masks in train_loader:
        images = images.to(device)
        masks = masks.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, masks)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, masks in val_loader:
            images = images.to(device)
            masks = masks.to(device)
            outputs = model(images)
            loss = loss_fn(outputs, masks)
            val_loss += loss.item()

    mean_iou, miou = evaluate_iou(model, val_loader, num_classes=3, device=device)
    print(f"Epoch {epoch+1}: Val Loss = {val_loss / len(val_loader):.4f}, mIoU = {miou:.4f}, Class IoUs = {mean_iou}")


Epoch 1: Val Loss = 0.5059, mIoU = 0.6708, Class IoUs = [0.96073165 0.17199674 0.87963269]
Epoch 2: Val Loss = 0.2404, mIoU = 0.7917, Class IoUs = [0.98218001 0.43212191 0.96093662]
Epoch 3: Val Loss = 0.1147, mIoU = 0.8187, Class IoUs = [0.98403956 0.50647311 0.96558464]
Epoch 4: Val Loss = 0.0739, mIoU = 0.8646, Class IoUs = [0.9872259  0.63212496 0.97432806]
Epoch 5: Val Loss = 0.0520, mIoU = 0.9048, Class IoUs = [0.98806015 0.74613982 0.98023589]
Epoch 6: Val Loss = 0.0483, mIoU = 0.8990, Class IoUs = [0.98794636 0.7303572  0.97869258]
Epoch 7: Val Loss = 0.0453, mIoU = 0.8968, Class IoUs = [0.98789616 0.72573359 0.97664895]
Epoch 8: Val Loss = 0.0467, mIoU = 0.8973, Class IoUs = [0.98398003 0.73250651 0.97553818]
Epoch 9: Val Loss = 0.0369, mIoU = 0.9127, Class IoUs = [0.98874945 0.76948062 0.97993303]
Epoch 10: Val Loss = 0.0320, mIoU = 0.9220, Class IoUs = [0.98903418 0.79454179 0.98255883]
Epoch 11: Val Loss = 0.0285, mIoU = 0.9235, Class IoUs = [0.98925998 0.79716571 0.9840673

In [ ]:
torch.save(model.state_dict(), 'unet_model_baru.pth')

In [ ]:
!pip install torchinfo -q
from torchinfo import summary

summary(model, input_size=(1, 3, 256, 256))  # batch size 1


Layer (type:depth-idx)                        Output Shape              Param #
Unet                                          [1, 3, 256, 256]          --
├─VGGEncoder: 1-1                             [1, 64, 256, 256]         --
│    └─Sequential: 2-1                        --                        --
│    │    └─Conv2d: 3-1                       [1, 64, 256, 256]         1,792
│    │    └─BatchNorm2d: 3-2                  [1, 64, 256, 256]         128
│    │    └─ReLU: 3-3                         [1, 64, 256, 256]         --
│    │    └─Conv2d: 3-4                       [1, 64, 256, 256]         36,928
│    │    └─BatchNorm2d: 3-5                  [1, 64, 256, 256]         128
│    │    └─ReLU: 3-6                         [1, 64, 256, 256]         --
│    │    └─MaxPool2d: 3-7                    [1, 64, 128, 128]         --
│    │    └─Conv2d: 3-8                       [1, 128, 128, 128]        73,856
│    │    └─BatchNorm2d: 3-9                  [1, 128, 128, 128]        256
│    │